In [ ]:
# Here are computing NMF decomposition for AD vs NC task

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
import pickle
from load_ADNI import load_ADNI, convert

In [2]:
path = '/nmnt/media/home/anvar/conferences_code/MICCAI2017/reproducing_overlappingMICCAI/data'
_, _, info = load_ADNI(path)

ADNI data shape                   : (807, 68, 68) 
ADNI target variable shape        : (807,) 
ADNI number of unique patients    : (255,)



Little preprocessing
- binarizing
- zerod diagonals
- selecting only AD and Normal phenotypes
- creating appropriate target vector and groups vector

In [6]:
data = pd.concat([info[info.target == 'AD'], info[info.target == 'Normal']])
data = data.sort_values(['target', 'subject_id', 'scan_id'],)

In [7]:
data.head()

,subject_id_file,subject_id,scan_id,matrix,target
subject_id_file,,,,,
003_S_4136_1,003_S_4136_1,003_S_4136,1,"[2618.0, 7.0, 68.0, 14.0, 3.0, 73.0, 1303.0, 4...",AD
003_S_4136_2,003_S_4136_2,003_S_4136,2,"[3620.0, 36.0, 50.0, 30.0, 9.0, 93.0, 1683.0, ...",AD
003_S_4136_3,003_S_4136_3,003_S_4136,3,"[3004.0, 3.0, 228.0, 3.0, 20.0, 120.0, 1741.0,...",AD
003_S_4136_4,003_S_4136_4,003_S_4136,4,"[3203.0, 0.0, 45.0, 19.0, 0.0, 115.0, 1683.0, ...",AD
003_S_4142_1,003_S_4142_1,003_S_4142,1,"[2450.0, 0.0, 401.0, 0.0, 0.0, 9.0, 1536.0, 93...",AD


In [8]:
data.groupby('target').count()

# 136 AD,  190 Normal, total of 326 scans/connectomes
# 108 unique subjects

,subject_id_file,subject_id,scan_id,matrix
target,,,,
AD,136,136,136,136
Normal,190,190,190,190


In [9]:
data.drop_duplicates('subject_id').shape

(108, 5)

In [10]:
matrices = np.zeros((326, 68, 68))
for i in range(326):
    matrices[i] = convert(data.matrix[i], diag=0)

In [17]:
# encode target 
target = data.target.map({'AD':0, 'EMCI':1, 'LMCI':2, 'Normal':3}).values

# encode subjects
sub_dict = dict(zip(np.unique(data.subject_id.values), np.arange(108)))
subjects = data.subject_id.map(sub_dict).values

# binarizing matrices
matrices[matrices>0] = 1

In [20]:
# Computing NMF decomposition

results = {}
results_full = {}
for n_clusters in range(2, 15):
    single_n = np.zeros((8, 326, n_clusters, 68))
    single_n_full = np.zeros((326, n_clusters, 68))
    for idx, mat in enumerate(matrices):
        nmf = NMF(n_components = n_clusters, random_state=1)
        W = nmf.fit_transform(mat)
        H = nmf.components_ / nmf.components_.sum(axis = 0)
        single_n_full[idx, :, :] = H
        for j, thresh in enumerate([.05, .10, .15, .20, .25, .30, .35, .40]):
            _H = np.where(H > thresh, 1, 0)
            single_n[j, idx, :, :] = _H
    results[n_clusters] = single_n
    results_full[n_clusters] = single_n_full

/nmnt/media/home/anvar/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
/nmnt/media/home/anvar/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in greater
  if sys.path[0] == '':


In [24]:
results_full[2].shape # <---- this contains 326 matrices of size 2 x 68, before thresholding

(326, 2, 68)

In [29]:
results_full[2][0].T # We have 68 nodes/vertices and two clusters, each column sums to 1
# i'th column has 2 elements, first element is strength of node i belongs to cluster 1,
# second element - strength of node i belongs to cluster 2

array([[ 0.81972294,  0.18027706],
       [ 1.        ,  0.        ],
       [ 1.        ,  0.        ],
       [ 0.61411749,  0.38588251],
       [ 1.        ,  0.        ],
       [ 0.9523918 ,  0.0476082 ],
       [ 0.75073357,  0.24926643],
       [ 0.8728751 ,  0.1271249 ],
       [ 0.78497572,  0.21502428],
       [ 0.93468836,  0.06531164],
       [ 0.91268785,  0.08731215],
       [ 0.72261117,  0.27738883],
       [ 0.75559643,  0.24440357],
       [ 0.87244462,  0.12755538],
       [ 1.        ,  0.        ],
       [ 0.76384522,  0.23615478],
       [ 1.        ,  0.        ],
       [ 0.8544608 ,  0.1455392 ],
       [ 0.82626225,  0.17373775],
       [ 0.69017051,  0.30982949],
       [ 1.        ,  0.        ],
       [ 0.77160942,  0.22839058],
       [ 1.        ,  0.        ],
       [ 0.72287456,  0.27712544],
       [ 0.9003915 ,  0.0996085 ],
       [ 0.72238749,  0.27761251],
       [ 0.58422421,  0.41577579],
       [ 0.81493427,  0.18506573],
       [ 0.75153099,

In [34]:
results[2].shape # <---- this contains 326 matrices of size 2 x 68, after thresholding

(8, 326, 2, 68)

In [42]:
results[2][0][0].T # As we can see sometimes i'th node belongs to both first and second cluster

array([[ 1.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  0.],
       [ 1.,  1.],
       [ 1.,  0.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  0.],
       [ 1.,  1.],
       [ 1.,  0.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  0.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 0.,  1.],
       [ 1.,  1.],
       [ 0.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 0.,  1.],
       [ 1.,  1.],
       [ 0.,

In [44]:
triu_grid = np.array(np.triu_indices(326, k = 1)).T
thresh = ['05', '10', '15', '20', '25', '30', '35', '40']
for n_clusters, group in results.items(): # for all number of clusters
    for idx1, partitions in enumerate(group): #  for all thresholds
        for idx2, partition in enumerate(partitions): # for all graphs in data
            path = 'NMF_results/{}_{}_{}'.format(n_clusters, thresh[idx1], idx2)
            f = open(path, 'w')
            for n, cluster in enumerate(partition):
                belonging_vertices = str(np.argwhere(cluster).flatten())[1:-1].split()
                for sym in belonging_vertices:
                    try:
                        int(sym)
                        f.write(sym+' ')
                    except:
                        pass
                if n < n_clusters:
                    f.write('\n')